In [73]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import random
import os
import json
import pandas as pd


In [109]:
head = 'https://www.indeed.com/'
data = get_jobs_of_title(head, job_titles,100)
df = pd.DataFrame(data)


,job_title,company,job_type,description,original_link
0,Data Scientist,"Technical Resources International, Inc.",Full-time,\n\nResponsibilities:\n\nManaging and sifting ...,https://www.indeed.com/viewjob?jk=ffb833c5937a...
1,Data Scientist,Q2ebanking,Full-time,\n\n Our Team\n \n\n The Data Science team ...,https://www.indeed.com/viewjob?jk=c10cfbb7f4f0...
2,Data Scientist,LOCKHEED MARTIN CORPORATION,nan,\n COVID-19 continues to significantly impact ...,https://www.indeed.com/viewjob?jk=44f2b2a37f2c...
3,Data Scientist/Principal Data Scientist,Northrop Grumman,- Part-time,\nRequisition ID: R10051536\n\nCategory: Resea...,https://www.indeed.com/viewjob?jk=1a3c492a422a...
4,Data Scientist - Analytics,Logic20/20 Inc.,- Full-time,\n Company Description\n We’re a six-time “Be...,https://www.indeed.com/viewjob?jk=bc66d5127b22...
5,Data Scientist,Comcentric,"Full-time, Contract",\nWe are seeking a REMOTE Data Scientist for a...,https://www.indeed.com/viewjob?jk=3f86089d249b...
6,SR DATA SCIENTIST MACHINE LEARNING,Johnson & Johnson,nan,\nJohnson & Johnson is recruiting for a Senior...,https://www.indeed.com/viewjob?jk=995c9edfd442...
7,Senior Data Scientist,Ripple,nan,\nWhat You’ll Do:\n\nSupport business units by...,https://www.indeed.com/viewjob?jk=58316778f71c...
8,Data Warehouse Engineer,Humanity,Full-time,\n Job Description\n \n\n Job Description\n \n...,https://www.indeed.com/viewjob?jk=1f54ee089db3...
9,"Hybrid Data Scientist, Senior",Booz Allen Hamilton,nan,\nThe Challenge:\n Are you excited at the pros...,https://www.indeed.com/viewjob?jk=f6c38f22a9dc...


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   job_title      46 non-null     object
 1   company        46 non-null     object
 2   job_type       46 non-null     object
 3   description    46 non-null     object
 4   original_link  46 non-null     object
dtypes: object(5)
memory usage: 1.9+ KB


In [112]:
df.to_csv('./indeed_DS_jobs.csv')

In [106]:
def get_jobs_of_title(head, job_title, jobs_per_page):
    """
    Args:
        job_title (str): example: 'data+scientist'

    Returns:
    """

    #needed to be changed
    num_pages = 1 #number of pages to scrape
    page_gap_min = 1 #min sleep time between pages
    page_gap_max = 3 #max sleep time between pages
    job_per_page = jobs_per_page #number of jobs in one page
    job_gap_min = 1 #min sleep time between jobs
    job_gap_max = 3 #max sleep time between jobs
    
    data = {"job_title":[],"company":[],'job_type':[],"description":[],"original_link":[]}
    for i in range(num_pages): 
        #sleep between each call
        gap = random.uniform(page_gap_min,page_gap_max) 
        time.sleep(gap)

        #each page contains 50 jobs
        tail = "jobs?q={0}&sort=date&limit={1}".format(job_title,job_per_page)
        if i>0:
            tail += "&start={0}".format(i*job_per_page)

        #get link to joblist page
        url = head+tail
         
        #get links to webpages of jobs on the joblist
        job_page_links = get_job_links_from_page(head,url)
        
        for job_page_link in job_page_links:
            soup = get_soup(job_page_link)
            gap = random.uniform(job_gap_min,job_gap_max) 
            time.sleep(gap)
   
            title = soup.find('h1', class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title')
            if not title:
                data["job_title"].append('nan')
            else:
                data["job_title"].append(title.getText())
            company = soup.find('a', target="_blank")
            if not company:
                data["company"].append('nan')
            else:
                data["company"].append(company.getText())
            jtype = soup.find('span',class_="jobsearch-JobMetadataHeader-item")
            if not jtype:
                data['job_type'].append('nan')
            else:
                data['job_type'].append(jtype.getText())
            descript = soup.find('div',class_='jobsearch-jobDescriptionText').find('div')
            if not descript:
                data["description"].append('nan')
            else:
                data["description"].append(descript.getText())
                data["original_link"].append(job_page_link)

    return data

def get_job_links_from_page(head, url):
    """
    This function gets the links of the jobs on the joblist page.

    Args:
        url (str): link to joblist page

    Returns:
        job_page_links (list): list of links to the webpages of the jobs
    """

    job_page_links = []
    soup = get_soup(url)
    for item in soup.find_all("a", href=True):
        if '/rc/clk?jk=' in str(item) and 'fccid=' in str(item):
            link = item['href'].split("clk?")[1]
            job_page_links.append(head+'viewjob?'+link)
    return job_page_links
